In [135]:
!pip install transformers

In [136]:
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F

In [137]:
import json
import requests

In [138]:
API_TOKEN = ""

In [139]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

### Zero-shot learning

In [150]:
sentence = '''Полузащитник «Манчестер Юнайтед» Бруну Фернандеш поделился мыслями после поражения от «Манчестер Сити» (0:2) в АПЛ. 
«Мы должны меньше говорить и больше делать. Я говорю одно и то же. Мы должны измениться, подобное уже происходило много-много раз. Того, что мы сделали, было недостаточно.
Если мы хотим быть на одном уровне с ними, то должны делать больше. Сейчас они показали, что мы не на одном уровне. Каждый должен смотреть на себя. Нужно глянуть в зеркало и увидеть, что ты делаешь неправильно.
Нельзя изменить характер кого-то другого. Ты должен посмотреть на себя и понять, в чем конкретно можешь прибавить», – сказал Бруну в эфире Sky Sports.'''

labels = ['спорт', 'новости']

In [151]:
tokenizer.pad_token = 0

In [152]:
# run inputs through model and mean-pool over the sequence
# dimension to get sequence-level representations
inputs = tokenizer.batch_encode_plus([sentence] + labels, return_tensors='pt', padding='longest')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
output = model(input_ids, attention_mask=attention_mask)[0]
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

# now find the labels with the highest cosine similarities to the sentence
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
for ind in closest:
    print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')

label: спорт 	 similarity: 0.6847456097602844
label: новости 	 similarity: 0.6765518188476562


### Few Short Learning

In [ ]:
def query(payload='',parameters=None,options={'use_cache': False}):
    API_URL = "https://api-inference.huggingface.co/models/sberbank-ai/rugpt3small_based_on_gpt2"
    headers = {"Authorization": f"Bearer {API_TOKEN}"}
    body = {"inputs":payload,'parameters':parameters,'options':options}
    response = requests.request("POST", API_URL, headers=headers, data=json.dumps(body))
    try:
      response.raise_for_status()
    except requests.exceptions.HTTPError:
        return "Error:"+" ".join(response.json()['error'])
    else:
      return response.json()[0]

In [167]:
parameters = {
    'max_new_tokens': 1,  # number of generated tokens
    'temperature': 0.5,   # controlling the randomness of generations
    'end_sequence': "###" # stopping sequence for generation
}

In [39]:
prompt="""Извлечь название клуба из предложения:

предложение: Челси в родных стенах уверенно победил в лондонском дерби
клуб: Челси
###
предложение: Барселона назначила Хави главным тренером команды 
клуб: Барселона
###
предложение: Зенит находится в хорошей форме
клуб: Зенит
###
предложение: Спартак никак не может найти свою игру и теряет очки в последних матчах
клуб: Спартак
###
предложение: Итальянский специалист возглавил Тоттенхем
клуб: Тоттенхем
###
предложение: Ньюкасл купили шейхи и будут проводить мощную трансферную компанию этой зимой
клуб: Ньюкасл
###
предложение: Ливерпуль грамотно использует прессинг
клуб:
"""

data = query(prompt, parameters)

In [40]:
print('Expected: Ливерпуль')
print('Predicted: ' + data['generated_text'].split('клуб:')[-1].lstrip())

Expected: Ливерпуль
Predicted: Ливер
